# Introduction to machine learning: classification of basalt source

## Import scientific python libraries

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import copy

## Machine learning
Text from: https://scikit-learn.org/stable/tutorial/basic/tutorial.html

In general, a learning problem considers a set of n samples of data and then tries to predict properties of unknown data. If each sample is more than a single number and, for instance, a multi-dimensional entry (aka multivariate data), it is said to have several attributes or features.

Learning problems fall into a few categories:
- **supervised learning**, in which the data comes with additional attributes that we want to predict (Click here to go to the scikit-learn supervised learning page).This problem can be either:
    - *classification*: samples belong to two or more classes and we want to learn from already labeled data how to predict the class of unlabeled data. An example of a classification problem would be handwritten digit recognition, in which the aim is to assign each input vector to one of a finite number of discrete categories. Another way to think of classification is as a discrete (as opposed to continuous) form of supervised learning where one has a limited number of categories and for each of the n samples provided, one is to try to label them with the correct category or class.
    - *regression*: if the desired output consists of one or more continuous variables, then the task is called regression. An example of a regression problem would be the prediction of the length of a salmon as a function of its age and weight.

- **unsupervised learning**, in which the training data consists of a set of input vectors x without any corresponding target values. The goal in such problems may be to discover groups of similar examples within the data, where it is called clustering, or to determine the distribution of data within the input space, known as density estimation, or to project the data from a high-dimensional space down to two or three dimensions for the purpose of visualization (Click here to go to the Scikit-Learn unsupervised learning page).

### Training set and testing set

Machine learning is about learning some properties of a data set and then testing those properties against another data set. A common practice in machine learning is to evaluate an algorithm by splitting a data set into two. We call one of those sets the training set, on which we learn some properties; we call the other set the testing set, on which we test the learned properties.

**Today we will focus on classification through a supervised learning approach**

*Systems doing this type of analysis are all around us. Consider a spam filter for example*

# Classifying volcanic rocks

<img src="./images/volcanic-tectonics.png" width = 600 align = 'center'>

Today we are going to continue dealing with igneous geochemistry data. Igneous rocks are those that crystallize from cooling magma. Different magmas have different compositions associated with their origin as we explore two weeks ago. During class today, we will continue to focus on data from mafic lava flows (these are called basalts and are the relatively low silica, high iron end of what we looked at two weeks ago).

> Igneous rocks form in a wide variety of tectonic settings,
including mid-ocean ridges, ocean islands, and volcanic
arcs. It is a problem of great interest to igneous petrologists
to recover the original tectonic setting of mafic rocks of the
past. When the geological setting alone cannot unambiguously
resolve this question, the chemical composition of
these rocks might contain the answer. The major, minor,
and trace elemental composition of basalts shows large
variations, for example as a function of formation depth
(e.g., Kushiro and Kuno, 1963) --- *Vermeesch (2006)*

For this analysis we are going to use a dataset that was compiled in 

Vermeesch (2006) Tectonic discrimination of basalts with classification trees, *Geochimica et Cosmochimica Acta*  https://doi.org/10.1016/j.gca.2005.12.016

These data were grouped into 3 categories:

- 256 ***Island arc basalts (IAB)*** from the Aeolian, Izu-Bonin, Kermadec, Kurile, Lesser Antilles, Mariana, Scotia, and Tonga arcs.
- 241 ***Mid-ocean ridge (MORB)*** samples from the East Pacific Rise, Mid Atlantic Ridge, Indian Ocean, and Juan de Fuca Ridge.
- 259 ***Ocean-island (OIB)*** samples from St. Helena, the Canary, Cape Verde, Caroline, Crozet, Hawaii-Emperor, Juan Fernandez, Marquesas, Mascarene, Samoan, and Society islands.

**Let's look at the illustration above and determine where each of these settings are within a plate tectonic context**

## Import data


The data are from the supplemental materials of the Vermeesch (2006) paper. The samples are grouped by affinity MORB, OIB, and IAB. They are additionally assigned affinity codes and colors from the default matplotlib cycle:

|affinity| affinity code | color |
|--------|---------------|-------|
| MORB| 0 | C0
| OIB |  1 | C1
| IAB |  2 | C2

In [ ]:
basalt_data = pd.read_csv('./data/Vermeesch2006.csv')
basalt_data.tail()

In [ ]:
MORB_data = basalt_data[basalt_data['affinity']=='MORB']
OIB_data = basalt_data[basalt_data['affinity']=='OIB']
IAB_data = basalt_data[basalt_data['affinity']=='IAB']

## Can geochemical data be used to classify the tectonic setting?

These data are labeled. The author already determined what setting these basalts came from. However, is there are way that we could use these labeled data to determine the setting for an unknown basalt?

A paper published in 1982 proposed that the elements titanium and vanadium were particular good at giving insight into tectonic setting. The details of why are quite complicated and can be summarized as "the depletion of V relative to Ti is a function of the fO2 of the magma and its source, the degree of partial melting, and subsequent fractional crystallization." If you take EPS100B you will learn more about the fundamentals behind this igneous petrology. *For the moment you can consider the working hypothesis behind this classification to that different magmatic environments have differences in oxidation states that are reflected in Ti vs V ratios.*

Shervais, J.W. (1982) Ti-V plots and the petrogenesis of modern and ophiolitic lavas *Earth and Planetary Science Letters* https://doi.org/10.1016/0012-821X(82)90120-0

### Plot TiO2 (wt%) vs V (ppm)

In [ ]:
plt.scatter(MORB_data['TiO2 (wt%)'],MORB_data['V (ppm)'],label='mid-ocean ridge',edgecolors='black')
plt.scatter(OIB_data['TiO2 (wt%)'],OIB_data['V (ppm)'],label='ocean island',edgecolors='black')
plt.scatter(IAB_data['TiO2 (wt%)'],IAB_data['V (ppm)'],label='island arc',edgecolors='black')
plt.xlabel('TiO2 (wt%)')
plt.ylabel('V (ppm)')
plt.legend()
plt.show()

scatter_plot = plt.scatter(basalt_data['TiO2 (wt%)'],basalt_data['V (ppm)'],color=basalt_data['color'],edgecolors='black')
plt.xlabel('TiO2 (wt%)')
plt.ylabel('V (ppm)')
plt.title('MORB (blue), OIB (orange), IAB (green)')
plt.show()

### Use the pandas groupby function to group by affinity and describe the values of one column

In [ ]:
basalt_data.groupby('affinity')['TiO2 (wt%)'].describe()

**CODE FOR YOU TO WRITE: Use the groupby command and describe the grouped vanadium concentration for the data.**

*Can we different between the different affinities on vanadium concentration alone?*

## Eye test classification method

In order to classify the basalt into their affinity based on titanium and vanadium concentrations, we can use a classification method.

The goal here is to be able to make an inference of what environment an unknown basalt formed in based on comparison to these data.

Let's say that we have two points where there affinity is unknown.
- point 1 has TiO2 of 4% and V concentration of 300 ppm
- point 2 has TiO2 of 1% and V concentration of 350 ppm
- point 3 has TiO2 of 1.9% and V concentration of 200 ppm

**Let's take votes on how they should be classified**

***WRITE HOW YOU THINK THEY SHOULD BE CLASSIFIED HERE***

In [ ]:
point_1_TiO2 = 4
point_1_V = 300
point_2_TiO2 = 1
point_2_V = 350
point_3_TiO2 = 1.9
point_3_V = 200

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(MORB_data['TiO2 (wt%)'],MORB_data['V (ppm)'],label='mid-ocean ridge',edgecolors='black')
plt.scatter(OIB_data['TiO2 (wt%)'],OIB_data['V (ppm)'],label='ocean island',edgecolors='black')
plt.scatter(IAB_data['TiO2 (wt%)'],IAB_data['V (ppm)'],label='island arc',edgecolors='black')
plt.scatter(point_1_TiO2,point_1_V,label='unknown point 1',color='black',marker='d',s=100)
plt.scatter(point_2_TiO2,point_2_V,label='unknown point 2',color='red',marker='>',s=100)
plt.scatter(point_3_TiO2,point_3_V,label='unknown point 2',color='yellow',edgecolors='black',marker='s',s=100)
plt.xlabel('TiO2 (wt%)')
plt.ylabel('V (ppm)')
plt.legend()
plt.show()

## Nearest Neighbors Classification

In nearest neighbors classification, classification is computed from a simple majority vote of the nearest neighbors of each point: a query point is assigned the data class which has the most representatives within the nearest neighbors of the point. There are different ways this can be done and can be weighted.

### Filter the data to ones that have Ti and V data

In [ ]:
basalt_data_Ti_V = basalt_data[(~basalt_data['TiO2 (wt%)'].isna()) & (~basalt_data['V (ppm)'].isna())]

### Normalize the data

Given that the nearest neighbor is a distance and that they y-axis and x-axis are so different (in part because of different units) we need to normalize the data. We will divide the 'TiO2 (wt%)' by the maximum 'TiO2 (wt%)' to get a value between 0 and 1. We will do the same for V (ppm) as well.

In [ ]:
basalt_data_Ti_V.loc[:,'Ti_norm'] = basalt_data['TiO2 (wt%)']/np.max(basalt_data['TiO2 (wt%)'])
basalt_data_Ti_V.head()

**Code for you to write**

Make a column called V_norm that is normalized vanadium.

In [ ]:
### INSTRUCTOR VERSION
basalt_data_Ti_V.loc[:,'V_norm'] = basalt_data['V (ppm)']/np.max(basalt_data['V (ppm)'])

**Code for you to write**

Make a scatter plot of Ti_norm vs V_norm that is colored by affinity (`color=basalt_data_Ti_V['color']`). It should look a lot like the previous scatter plots.

In [ ]:
### INSTRUCTOR VERSION
scatter_plot = plt.scatter(basalt_data_Ti_V['Ti_norm'],basalt_data_Ti_V['V_norm'],color=basalt_data_Ti_V['color'],edgecolors='black')
plt.xlabel('Ti_norm')
plt.ylabel('V_norm')
plt.title('MORB (blue), OIB (orange), IAB (green)')
plt.show()

### Preparing arrays of the data

We will make a 2 x n array of the TiO2 (wt%) and V (ppm) values (where n is the number of data points) and a 1 x n array of the classifications (the tectonic affinities).

In [ ]:
basalt_Ti_V = basalt_data_Ti_V[['Ti_norm', 'V_norm']].values
basalt_Ti_V

In [ ]:
basalt_affinity = basalt_data_Ti_V['affinity'].tolist()
basalt_affinity_code = basalt_data_Ti_V['affinity code'].tolist()

### Import sci-kit learn 

We will be using the scikit-learn library which is a widely used library for machine learning in Python.

In [ ]:
from sklearn import neighbors

### Define our classifier

We will construct a classifier that uses the 5 nearest neighbors (`n_neighbors=5`) and weight points by the inverse of their distance (`weights='distance'`) such that closer neighbors of a query point will have a greater influence than neighbors which are further away.

In [ ]:
classifier_neighbors = neighbors.KNeighborsClassifier(n_neighbors=10, weights='distance')

### Fit/train the classifier

We can then feed the array of the data and the array of the classification in a `.fit` function preformed on the classifier object.

In [ ]:
classifier_neighbors.fit(basalt_Ti_V, basalt_affinity)

### Normalize our mystery points

In [ ]:
point_1_TiO2_norm = 4/np.max(basalt_data['TiO2 (wt%)'])
point_1_V_norm = 300/np.max(basalt_data['V (ppm)'])
point_2_TiO2_norm = 1/np.max(basalt_data['TiO2 (wt%)'])
point_2_V_norm = 350/np.max(basalt_data['V (ppm)'])
point_3_TiO2_norm = 1.9/np.max(basalt_data['TiO2 (wt%)'])
point_3_V_norm = 200/np.max(basalt_data['V (ppm)'])

In [ ]:
plt.figure(figsize=(6,6))
scatter_plot = plt.scatter(basalt_data_Ti_V['Ti_norm'],basalt_data_Ti_V['V_norm'],color=basalt_data_Ti_V['color'],edgecolors='black')
plt.scatter(point_1_TiO2_norm,point_1_V_norm,label='unknown point 1',color='black',marker='d',s=100)
plt.scatter(point_2_TiO2_norm,point_2_V_norm,label='unknown point 2',color='red',marker='>',s=100)
plt.scatter(point_3_TiO2_norm,point_3_V_norm,label='unknown point 2',color='yellow',edgecolors='black',marker='s',s=100)
plt.xlabel('Ti_norm')
plt.ylabel('V_norm')
plt.gca().set_aspect('equal', 'box')
plt.title('MORB (blue), OIB (orange), IAB (green)')
plt.legend()
plt.show()

### Predict the tectonic affinity of the mystery points using the neighbors classifier

In [ ]:
classifier_neighbors.predict([[point_1_TiO2_norm,point_1_V_norm],
                             [point_2_TiO2_norm,point_2_V_norm],
                             [point_3_TiO2_norm,point_3_V_norm]])

### Fit/train using the basalt_affinity_code rather than the string names

In [ ]:
classifier_neighbors.fit(basalt_Ti_V, basalt_affinity_code)

In [ ]:
classifier_neighbors.predict([[point_1_TiO2_norm,point_1_V_norm],
                    [point_2_TiO2_norm,point_2_V_norm],
                    [point_3_TiO2_norm,point_3_V_norm]])

### Visualizing the decision boundary

Let's make a 101 x 101 grid of x and y values between 0 and 1.

In [ ]:
xx, yy = np.meshgrid(np.linspace(0, 1, 101),
                     np.linspace(0, 1, 101))
grid = np.c_[xx.ravel(), yy.ravel()]

In [ ]:
plt.scatter(xx, yy, s=1)
plt.tight_layout()

### Classify the grid

In [ ]:
grid_classes = classifier_neighbors.predict(grid)
grid_classes

In [ ]:
cmap = ListedColormap(['C0', 'C1', 'C2'])
grid_classes = grid_classes.reshape(xx.shape)

plt.figure(figsize=(6,6))

plt.pcolormesh(xx, yy, grid_classes, cmap=cmap)
scatter_plot = plt.scatter(basalt_data_Ti_V['Ti_norm'],basalt_data_Ti_V['V_norm'],
                           color=basalt_data_Ti_V['color'],edgecolors='black')

plt.scatter(point_1_TiO2_norm,point_1_V_norm,label='unknown point 1',color='black',marker='d',s=100)
plt.scatter(point_2_TiO2_norm,point_2_V_norm,label='unknown point 2',color='red',marker='>',s=100)
plt.scatter(point_3_TiO2_norm,point_3_V_norm,label='unknown point 2',color='yellow',edgecolors='black',marker='s',s=100)

plt.xlabel('Ti_norm')
plt.ylabel('V_norm')
plt.xlim(0,1)
plt.ylim(0,1)
plt.gca().set_aspect('equal', 'box')
plt.show()

## Training and testing

How good is our nearest neighbor classifier? To answer this we'll need to find out how frequently our classifications are correct.

**Discussion question**

*How should be determine the accuracy of this classification scheme using the data that we already have?*

In [ ]:
len(basalt_data_Ti_V)

### Making a training and testing data set

There are 514 rows with data. Let's use a random half of them for training and the other half for testing. To do this, we'll shuffle all the rows, take the first 257 as the training set, and the remaining 257 for testing.

In [ ]:
# Make a randomly ordered dataframe from the initial one
randomized_basalt_data = basalt_data_Ti_V.sample(frac=1) 

# Take the first 257 data points to use for "training"
training_data = copy.deepcopy(randomized_basalt_data.iloc[0:257])

# Use the rest to apply our machine learning on
remaining_data = copy.deepcopy(randomized_basalt_data.iloc[257:])

In [ ]:
basalt_Ti_V_training = training_data[['Ti_norm', 'V_norm']].values
basalt_Ti_V_remaining = remaining_data[['Ti_norm', 'V_norm']].values
basalt_affinity_training = training_data['affinity code'].tolist()

In [ ]:
classifier_neighbors.fit(basalt_Ti_V_training, basalt_affinity_training)

### Visualize the classification regions fit with half the data

We can send the grid to the classifier to see the classification regions and decision boundary that has been fit with half of the data.

In [ ]:
grid_classes = classifier_neighbors.predict(grid)
grid_classes = grid_classes.reshape(xx.shape)

In [ ]:
plt.figure(figsize=(6,6))
plt.pcolormesh(xx, yy, grid_classes, cmap=cmap)
plt.xlabel('Ti_norm')
plt.ylabel('V_norm')
plt.xlim(0,1)
plt.ylim(0,1)
plt.gca().set_aspect('equal', 'box')
plt.show()

### Compare the remaining data (test data) to the classification regions

Place the test data on this graph and you can see at once that while the classifier got many of the points right, there are some mis-classified points.

In [ ]:
plt.figure(figsize=(6,6))
plt.pcolormesh(xx, yy, grid_classes, cmap=cmap)

plt.scatter(remaining_data['Ti_norm'],remaining_data['V_norm'],
                           color=remaining_data['color'],edgecolors='black')

plt.xlabel('Ti_norm')
plt.ylabel('V_norm')
plt.xlim(0,1)
plt.ylim(0,1)
plt.gca().set_aspect('equal', 'box')
plt.show()

### Estimating the accuracy of the classifier

Since the test set was chosen randomly from the original sample it should preform with similar accuracy on the overall population. Let's calculate the success rate of the classification.

We will input the remaining data (test data) to the classifier and then assign these classified affinities to a new column in pandas.

In [ ]:
remaining_classes = classifier_neighbors.predict(basalt_Ti_V_remaining)

In [ ]:
remaining_data['predicted_class'] = remaining_classes

In [ ]:
remaining_data.head()

Now we have a new column of the classified affinities for the test data. We also have the actually affinities given that the data were originally labeled with classifications.

In [ ]:
remaining_data['correct_assignment'] = remaining_data['predicted_class'].eq(remaining_data['affinity code'])
remaining_data.head()

In [ ]:
remaining_data['correct_assignment'].value_counts(normalize=True) * 100

### Using scikit-learn functions to get an accuracy score of this nearest neighbor approach

Given that this approach of randomly splitting the data into training and test groups is quite common in macine learning classification, there are built-in convenience functions that can be used to more compactly do the same operations that we did above.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# split the data with 50% in each set
X1, X2, y1, y2 = train_test_split(basalt_Ti_V, basalt_affinity_code,train_size=0.5)

# fit the model on one set of data
classifier_neighbors.fit(X1, y1)

# evaluate the model on the second set of data
y2_model = classifier_neighbors.predict(X2)
accuracy_score(y2, y2_model)

## Other classification algorithms

If you go to the scikit-learn homepage you will find many available classifiers: https://scikit-learn.org/stable/index.html. They are nicely illustrated in this code from the scikit-learn documentation.

In [ ]:
# Code source: Gaël Varoquaux
#              Andreas Müller
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

h = .02  # step size in the mesh

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

X, y = make_classification(n_features=2, n_redundant=0, n_informative=2,
                           random_state=1, n_clusters_per_class=1)
rng = np.random.RandomState(2)
X += 2 * rng.uniform(size=X.shape)
linearly_separable = (X, y)

datasets = [make_moons(noise=0.3, random_state=0),
            make_circles(noise=0.2, factor=0.5, random_state=1),
            linearly_separable
            ]

figure = plt.figure(figsize=(27, 9))
i = 1
# iterate over datasets
for ds_cnt, ds in enumerate(datasets):
    # preprocess dataset, split into training and test part
    X, y = ds
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=.4, random_state=42)

    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    # just plot the dataset first
    cm = plt.cm.RdBu
    cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
    if ds_cnt == 0:
        ax.set_title("Input data")
    # Plot the training points
    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
               edgecolors='k')
    # Plot the testing points
    ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6,
               edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xticks(())
    ax.set_yticks(())
    i += 1

    # iterate over classifiers
    for name, clf in zip(names, classifiers):
        ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)

        # Plot the decision boundary. For that, we will assign a color to each
        # point in the mesh [x_min, x_max]x[y_min, y_max].
        if hasattr(clf, "decision_function"):
            Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
        else:
            Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]

        # Put the result into a color plot
        Z = Z.reshape(xx.shape)
        ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)

        # Plot the training points
        ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
                   edgecolors='k')
        # Plot the testing points
        ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright,
                   edgecolors='k', alpha=0.6)

        ax.set_xlim(xx.min(), xx.max())
        ax.set_ylim(yy.min(), yy.max())
        ax.set_xticks(())
        ax.set_yticks(())
        if ds_cnt == 0:
            ax.set_title(name)
        ax.text(xx.max() - .3, yy.min() + .3, ('%.2f' % score).lstrip('0'),
                size=15, horizontalalignment='right')
        i += 1

plt.tight_layout()
plt.show()

As a word of warning, we shouldn't get too carried away. Clearly, there are complexities related to this approach (our accuracy scores aren't that high). Shervais notes that: 
> "More specific evaluation of the tectonic setting of these and other ophiolites requires
application of detailed geologic and petrologic data as well as geochemistry. The Ti/V discrimination diagram, however,
is a potentially powerful adjunct to these techniques."

Additionally, we would like to be able to assign physical processes to the classification.

In [ ]:
scatter_plot = plt.scatter(basalt_data['TiO2 (wt%)'],basalt_data['V (ppm)'],color=basalt_data['color'],edgecolors='black')
plt.xlabel('TiO2 (wt%)')
plt.ylabel('V (ppm)')
plt.title('MORB (blue), OIB (orange), IAB (green)')
plt.show()

## Implementing a linear classifier

Rather than using a nearest neighbor approach we could instead implement hard cut offs as lines using a linear classifier. A benefit of using such a classifier is that the data do not need to be normalized between 0 and 1. Instead, the actual values can be used.

In [ ]:
from sklearn.svm import SVC
classifier_svc = SVC(kernel="poly",degree=1)

In [ ]:
basalt_Ti_V_unnorm = basalt_data_Ti_V[['TiO2 (wt%)','V (ppm)']].values

**Code for you to write: implement the classifier_svc and determine its accuracy using a training set and a test set**

**Code for you to write: pass a grid to the classifier and plot with the data**

We will want to use a grid that is spaced according to the non-normalized data.

In [ ]:
xx, yy = np.meshgrid(np.linspace(0, 5, 101),
                     np.linspace(0, 600, 101))
grid = np.c_[xx.ravel(), yy.ravel()]